PART 1

In [0]:
import nltk
nltk.download('punkt')
import numpy as np
from collections import Counter
import itertools
from tqdm import tqdm_notebook as tqdm
import time

In [0]:
#Read the text into memory, tokenize the text into words
def tokenize(file_path):  
    data, lst = [], []
    with open(file_path, 'r') as f:  
        for line in f:
            if line== '\n':
                data.append(lst)
                lst = []
            else:
                lines = line.replace("\n",'').split(" ")
                lst.append(tuple(lines))
    return data

EN_train_sentences = tokenize('data/EN/train')
#print(EN_train_sentences)      


In [3]:
def emission_parameter(data):
    
    tag_count = {}
    match_count = {}
    all_words, all_tags = [], []
    
    for lst in data:         
        for wordTuples in lst:
            
            tag = wordTuples[1]
            all_tags.append(tag)
            all_words.append(wordTuples[0])
            tag_count[tag] = tag_count.get(tag, 0) + 1
            match_count[wordTuples] = match_count.get(wordTuples, 0) + 1
            
      
    em = {i: np.log(match_count[i]/tag_count[i[1]]) for i in match_count}
    word_count = dict(Counter(all_words))
    all_words, all_tags = list(set(all_words)), list(set(all_tags))
    for p in range(len(all_words)):
        for q in range(len(all_tags)):       
            if (all_words[p], all_tags[q]) not in em:
                em[(all_words[p], all_tags[q])] = -2**31
    
    return word_count, tag_count, em, match_count, all_tags
  
EN_train_word_count, EN_train_tag_count, EN_train_emission, EN_emission_count, EN_all_tags = emission_parameter(EN_train_sentences)

print(EN_all_tags)
print(EN_train_word_count)
print(EN_train_tag_count)
print(EN_train_emission)
print(EN_emission_count)

['O', 'I-negative', 'B-positive', 'I-positive', 'B-neutral', 'B-negative', 'I-neutral']
{'All': 3, 'in': 267, 'all': 78, ',': 965, 'the': 1132, 'food': 259, 'was': 444, 'great': 152, '(': 87, 'except': 4, 'for': 287, 'dessserts': 1, ')': 92, '.': 1508, 'I': 485, 'have': 157, 'NEVER': 5, 'been': 53, 'disappointed': 13, 'Red': 2, 'Eye': 1, 'Great': 35, 'with': 155, 'an': 53, 'awesome': 7, 'atmosphere': 30, '!': 280, 'The': 312, 'sangria': 1, 'pretty': 16, 'tasty': 20, 'and': 775, 'good': 144, 'on': 160, 'a': 543, 'hot': 14, 'muggy': 1, 'day': 14, 'Also': 9, 'waiters': 9, 'try': 36, 'to': 504, 'push': 1, 'more': 43, 'you': 196, 'like': 69, 'suggest': 2, 'things': 12, 'as': 66, 'if': 59, 'they': 75, 'are': 126, 'complimentary': 5, 'when': 43, 'actually': 12, 'cost': 4, '$': 46, 'restaurant': 115, 'is': 465, 'cute': 6, 'but': 183, 'not': 155, 'upscale': 1, 'Hey': 1, 'think': 18, '2': 15, '+': 5, '5': 10, 'block': 3, 'walk': 5, "ain't": 1, 'bad': 26, 'And': 14, 'evaluated': 1, 'those': 9, 't

In [4]:
# Create a dictionary f, inside which we store the following key-value mappings:
# str(x, y)) -> log e(x|y)
def emission_parameter_string_key(data):
    ret = {}
    for k, v in data.items():
        ret['emission:'+ k[1]+ '+'+ k[0]] = v
    return ret

EN_emission_pairs = emission_parameter_string_key(EN_train_emission)
print(EN_emission_pairs)

{'emission:O+All': -9.017685611042436, 'emission:O+in': -4.54034879656423, 'emission:O+all': -5.785564559424215, 'emission:O+,': -3.24728344904484, 'emission:O+the': -3.0916488692569097, 'emission:B-positive+food': -2.2107041411644595, 'emission:O+was': -4.020473337278322, 'emission:O+great': -5.09241737886427, 'emission:O+(': -5.759589073020955, 'emission:O+except': -8.730003538590656, 'emission:O+for': -4.46732366154934, 'emission:B-negative+dessserts': -5.934894195619588, 'emission:O+)': -5.685481100867233, 'emission:O+.': -2.797758351142645, 'emission:O+I': -3.932149008773063, 'emission:O+have': -5.060052094362239, 'emission:O+NEVER': -8.506859987276446, 'emission:O+been': -6.146005986158425, 'emission:O+disappointed': -7.55134854224901, 'emission:B-positive+Red': -6.392754283805666, 'emission:I-positive+Eye': -6.3561076606958915, 'emission:O+Great': -6.560949838221132, 'emission:O+with': -5.13269127800221, 'emission:O+an': -6.146005986158425, 'emission:O+awesome': -8.1703877506552

PART 1 (ii)

In [5]:
def transition_parameter(data):
    tag_transition_count={} # (tag0,tag1):count
    
    tag_count = {} # tag0:count
    all_tags = []
    print(len(data))
    flag = 0
    
    for sent in data:
        idx = data.index(sent)
        
        for i in range(len(sent)):
            tag0 = sent[i-1][1] if i != 0 else 'START'
            tag1 = sent[i][1]
            tag_count[tag0]=tag_count.get(tag0,0) + 1
            tag_transition=(tag0,tag1)
            all_tags.append(tag0)
            tag_transition_count[tag_transition] = tag_transition_count.get(tag_transition,0)+1

        tag0 = sent[-1][1]
        tag1 = "STOP"
        tag_count[tag0]=tag_count.get(tag0,0) + 1
        tag_transition=(tag0,tag1)
        all_tags.append(tag0)
        tag_transition_count[tag_transition] = tag_transition_count.get(tag_transition,0)+1
                
    transition_parameter = {k: np.log(tag_transition_count[k]/tag_count[k[0]])
                        for k in tag_transition_count}
    
    all_tags = list(set(all_tags)) + ['STOP']
    print(all_tags)
    for p in range(len(all_tags)):
        for q in range(len(all_tags)):
            if (all_tags[p], all_tags[q]) not in transition_parameter:
                transition_parameter[(all_tags[p], all_tags[q])] = -2**31
    print(tag_count)
    
    return transition_parameter, tag_transition_count

EN_train_transition, EN_transition_count = transition_parameter(EN_train_sentences)
print(EN_train_transition)
print(EN_transition_count)

1873
['O', 'I-negative', 'B-positive', 'I-positive', 'START', 'B-neutral', 'B-negative', 'I-neutral', 'STOP']
{'START': 1873, 'O': 24743, 'B-positive': 1195, 'B-negative': 378, 'I-positive': 576, 'B-neutral': 77, 'I-neutral': 25, 'I-negative': 127}
2065
['O', 'B-conflict', 'I-negative', 'B-positive', 'I-positive', 'START', 'B-neutral', 'B-negative', 'I-neutral', 'STOP']
{'START': 2065, 'O': 32234, 'B-negative': 436, 'I-negative': 227, 'B-positive': 1288, 'B-neutral': 86, 'I-positive': 406, 'I-neutral': 23, 'B-conflict': 1}
{('START', 'O'): -0.06165959394788234, ('O', 'O'): -0.1469289780609056, ('O', 'B-positive'): -3.100585479223317, ('B-positive', 'O'): -0.3572728512809094, ('O', 'B-negative'): -4.235764913309846, ('B-negative', 'O'): -0.2344506222289012, ('O', 'STOP'): -2.5966054955940074, ('B-positive', 'I-positive'): -1.2540189870820944, ('I-positive', 'O'): -0.5419771288708248, ('O', 'B-neutral'): -5.94191062981491, ('B-neutral', 'O'): -0.23293155768037255, ('START', 'B-negative')

In [6]:
def transition_parameter_string_key(data):
    ret = {}
    for k, v in data.items():
        ret['transition:'+ k[0]+ '+'+ k[1]] = v
    return ret

EN_transition_pairs = transition_parameter_string_key(EN_train_transition)
print(EN_transition_pairs)

{'transition:START+O': -0.06165959394788234, 'transition:O+O': -0.1469289780609056, 'transition:O+B-positive': -3.100585479223317, 'transition:B-positive+O': -0.3572728512809094, 'transition:O+B-negative': -4.235764913309846, 'transition:B-negative+O': -0.2344506222289012, 'transition:O+STOP': -2.5966054955940074, 'transition:B-positive+I-positive': -1.2540189870820944, 'transition:I-positive+O': -0.5419771288708248, 'transition:O+B-neutral': -5.94191062981491, 'transition:B-neutral+O': -0.23293155768037255, 'transition:START+B-negative': -4.539564428890097, 'transition:START+B-positive': -3.153270067770207, 'transition:I-positive+I-positive': -0.8965221465517323, 'transition:B-neutral+I-neutral': -1.7788560643921472, 'transition:I-neutral+I-neutral': -0.7339691750802004, 'transition:I-neutral+O': -0.6539264674066639, 'transition:B-negative+I-negative': -1.6041608553332567, 'transition:I-negative+O': -0.5134537461722601, 'transition:START+B-neutral': -5.0503900526560885, 'transition:I-

PART 2 (i)

In [7]:
def calculate_score(sentence, emission, transition):
    wf = 0
    x_seq = [x[0] for x in sentence]
    y_seq = ['START']+[y[1] for y in sentence]+['STOP']
    
    for i in range(len(x_seq)):
        key = (x_seq[i], y_seq[i+1])
        wf+= emission[key]
    for j in range(1, len(y_seq)):
        key = (y_seq[j-1], y_seq[j])
        wf+= transition[key]
        
    return wf
calculate_score(EN_train_sentences[0], EN_train_emission, EN_train_transition)

-85.52845366888087

PART 2 (ii)

In [0]:
def viterbi(sentence, tags, trans, emit):    
    pi = [{}]
    path = {}
    
    # j = 0 (START)
    for tag in tags:
        pi[0][tag] = trans.get(('START',tag),0) + emit.get((sentence[0][0],tag),0)
        path[tag] = [tag]

    # j = 1 ~ N-1
    for t in range(1,len(sentence)):
        pi.append({})
        newpath = {}

        for y in tags:
            (prob, tag) = max([(pi[t-1][y0] + trans.get((y0,y),0) + emit.get((sentence[t][0],y),0), y0) 
                               for y0 in tags])
            pi[t][y] = prob
            newpath[y] = path[tag] + [y]

        #print(newpath)
        path = newpath
        
    # j = N+1 (STOP)
    (prob, tag) = max([(pi[len(sentence)-1][y] + trans.get((y,'STOP'),0), y) for y in tags])
    
    return prob, path[tag]

def write_prediction(output_filename, test_data, tags, emit, trans):
    
    with open(output_filename, "w") as outfile:
        for sentence in test_data:
            
            # generate tag sequence
            predicted_tag_sequence = viterbi(sentence, tags, trans, emit)[1]
            
            # handle inconsistent length
            if len(sentence) != len(predicted_tag_sequence):
                print("WARNING!! Different length {} / {}".format(sentence, predicted_tag_sequence))
            
            # write to file
            for i in range(len(sentence)):
                line = "{} {}\n".format(sentence[i][0], predicted_tag_sequence[i])
                outfile.write(line)
                
            outfile.write("\n")


In [9]:
EN_dev_sentences = tokenize('data/EN/dev.in')
print(EN_train_emission)

write_prediction('data/EN/dev.p2.out', EN_dev_sentences,
                 EN_all_tags, EN_train_emission, EN_train_transition)

{('All', 'O'): -9.017685611042436, ('in', 'O'): -4.54034879656423, ('all', 'O'): -5.785564559424215, (',', 'O'): -3.24728344904484, ('the', 'O'): -3.0916488692569097, ('food', 'B-positive'): -2.2107041411644595, ('was', 'O'): -4.020473337278322, ('great', 'O'): -5.09241737886427, ('(', 'O'): -5.759589073020955, ('except', 'O'): -8.730003538590656, ('for', 'O'): -4.46732366154934, ('dessserts', 'B-negative'): -5.934894195619588, (')', 'O'): -5.685481100867233, ('.', 'O'): -2.797758351142645, ('I', 'O'): -3.932149008773063, ('have', 'O'): -5.060052094362239, ('NEVER', 'O'): -8.506859987276446, ('been', 'O'): -6.146005986158425, ('disappointed', 'O'): -7.55134854224901, ('Red', 'B-positive'): -6.392754283805666, ('Eye', 'I-positive'): -6.3561076606958915, ('Great', 'O'): -6.560949838221132, ('with', 'O'): -5.13269127800221, ('an', 'O'): -6.146005986158425, ('awesome', 'O'): -8.170387750655234, ('atmosphere', 'B-positive'): -4.09016919081162, ('!', 'O'): -4.481508296541297, ('The', 'O'): -

PART 3 (i)

In [0]:
def logSumExp(ns):
    maxi = np.max(ns)
    ds = ns - maxi
    sumOfExp = np.exp(ds).sum()
    return maxi + np.log(sumOfExp)

def forward_algorithm(sentence, tags, trans, emit):    
    pi = [{}]
    
    # t = 1
    for y in tags:
        pi[0][y] = trans.get(('START',y),0) + emit.get((sentence[0][0], y), 0)

    # t = 2 ~ N
    for t in range(1,len(sentence)):
        pi.append({})

        for y in tags:
            log_alpha = []
            for y_prev in tags:
                log_alpha.append(pi[t-1][y_prev] + trans.get((y_prev,y),0) + emit.get((sentence[t][0],y),0))
            pi[t][y] = logSumExp(log_alpha)
            
    # t = N+1 (STOP)
    t = len(sentence)
    log_alpha = []
    for y in tags:
        log_alpha.append(pi[t-1][y] + trans.get((y,'STOP'),0))
    
    return pi, logSumExp(log_alpha)

#forward_algorithm(EN_train_sentences[0], EN_all_tags, EN_train_transition, EN_train_emission)

In [11]:
def loss_func(sentences, trans, emi, all_tags):
    loss = 0
    for sent in sentences:
        loss+= calculate_score(sent, emi, trans)
        _, update = forward_algorithm(sent, all_tags, trans, emi)
        loss-= update
    return (-1)*loss
loss_func(EN_train_sentences, EN_train_transition, EN_train_emission, EN_all_tags)

2050.7405338353574

PART 3 (ii)

In [0]:
def backward_algorithm(sentence, tags, trans, emit):    
    pi = [{} for i in range(len(sentence))]
    
    # t = N
    for tag in tags:
        pi[len(sentence)-1][tag] = trans.get((tag,'STOP'),0)
        

    # t = N-1 ~ 1
    for t in reversed(range(0,len(sentence)-1)):
        for y in tags:
            log_beta = []
            for y_next in tags:
                log_beta.append(pi[t+1][y_next] + trans.get((y, y_next),0) + emit.get((sentence[t+1][0],y_next),0))
            pi[t][y] = logSumExp(log_beta)

    # t = 0 (START)
    log_beta = []
    for y in tags:
        log_beta.append(pi[0][y] + trans.get(('START',y),0) + emit.get((sentence[0][0],y),0))
    
    return pi, logSumExp(log_beta)

#backward_algorithm(EN_train_sentences[0], EN_all_tags, EN_train_transition, EN_train_emission)

In [0]:
def fwd_bkw(sentence, all_tags, trans, emit):
    fwd, p_fwd = forward_algorithm(sentence, all_tags, trans, emit)
    bkw, p_bkw = backward_algorithm(sentence, all_tags, trans, emit)
    
        
    feature_expectation = {} # {feature: expectation}
    # t = 1
    for y in all_tags:
        x = sentence[0][0]
        t_feature = ('START',y)
        
        # update transition features
        update = bkw[0][y] + trans[t_feature] + emit[(x,y)] - p_fwd     
        feature_expectation[t_feature] = feature_expectation.get(t_feature,0) + np.exp(update)
        
        # update emission features
        e_feature = (x,y)
        update = fwd[0][y] + bkw[0][y] - p_fwd
        feature_expectation[e_feature] = feature_expectation.get(e_feature,0) + np.exp(update)

        
    # t = 2 ~ N
    for t in range(1,len(sentence)):
        for y in all_tags:
            x = sentence[t][0]
            x_prev = sentence[t-1][0]
            
            # update emission features
            e_feature = (x,y)
            update = fwd[t][y] + bkw[t][y] - p_fwd            
            feature_expectation[e_feature] = feature_expectation.get(e_feature,0) + np.exp(update)
            
            for y_prev in all_tags:
                # update transition features
                t_feature = (y_prev,y)
                update = fwd[t-1][y_prev] + bkw[t][y] + trans[t_feature] + emit[(x,y)]- p_fwd
                feature_expectation[t_feature] = feature_expectation.get(t_feature,0) + np.exp(update)
                
    # t = N+1 (STOP)
    t = len(sentence)
    for y in all_tags:
        # update transition features
        t_feature = (y,'STOP')
        update = fwd[t-1][y] + trans[t_feature]- p_fwd
        feature_expectation[t_feature] = feature_expectation.get(t_feature,0) + np.exp(update)
        
    return feature_expectation
                
#fwd_bkw(EN_train_sentences[0], EN_all_tags, EN_train_transition, EN_train_emission)

In [0]:
def get_feat_exp(ES_train_sentences, all_tags, trans, emi):
    feat_exp = {}
    for key in trans:
        feat_exp[key] = 0
    for key in emi:
        feat_exp[key] = 0

    for sent in ES_train_sentences:
        expect = fwd_bkw(sent, all_tags, trans, emi)
        for k,v in expect.items():
            
            feat_exp[k] += v
    return feat_exp
  
#print(get_feat_exp(EN_train_sentences, EN_all_tags, EN_train_transition, EN_train_emission))

In [0]:
def mapping(emission, transition):
    index_mapping = {}
    i = 0
    for em in emission:
        index_mapping[i] = em
        i+= 1
    for tr in transition:
        index_mapping[i] = tr
        i+= 1
        
    return index_mapping

index_mapping = mapping(EN_train_emission, EN_train_transition)

In [0]:
#feat_exp : []
def compute_grad(feat_exp, all_tags, trans, emi):
    grad_lst= [0 for i in range(len(index_mapping))]
    for i in range(len(feat_exp)):
        if index_mapping[i] in EN_emission_count:
            grad_lst[i] += (feat_exp[i]- EN_emission_count[index_mapping[i]])
        elif index_mapping[i] in EN_transition_count:
            grad_lst[i] += (feat_exp[i]- EN_transition_count[index_mapping[i]])
        else:
            grad_lst[i] += feat_exp[i]
    return grad_lst

#len(compute_grad(feat_exp, EN_all_tags, EN_train_sentences, EN_train_emission))

PART 4 

In [0]:
def loss_with_reg(w, sentences, trans, emi, all_tags, n = 0.1):
    loss = loss_func(sentences, trans, emi, all_tags)
    # regularization
    loss += n*sum(w1*w1 for w1 in w)
    return loss

In [0]:
def grad_with_reg(feat_exp, w, trans, emi, all_tags, n = 0.1):
    grad_lst = compute_grad(feat_exp, all_tags, trans, emi)
    for i in range(len(w)):
        grad_lst[i] += w[i]*2*n
    return grad_lst

In [19]:
from scipy.optimize import fmin_l_bfgs_b
import time
total_start = time.time()

def callbackF(w):
    '''
    This function will be called by "fmin_l_bfgs_b"
    Arg:
    w: weights, numpy array
    '''
    loss = get_loss_grad(w)[0]
    print('Loss:{0:.4f}'.format(loss))
def get_loss_grad(w):
    '''
    This function will be called by "fmin_l_bfgs_b"
    Arg:
    w: weights, numpy array
    Returns:
    loss: loss, float
    grads: gradients, numpy array
    '''
    # to be completed by you
    start = time.time()
    emi, trans = {}, {}
    for i in range(len(list(index_mapping.keys()))): #27899
        if i< len(EN_train_emission): #27818
            emi[index_mapping[i]] = w[i]
        else:
            trans[index_mapping[i]] = w[i]
    
    feat_exp = get_feat_exp(EN_train_sentences, EN_all_tags, trans, emi)
    # change feat_exp to list
    ft = []
    for i in range(len(list(index_mapping.keys()))):
        ft.append(feat_exp[index_mapping[i]])
    
    loss = loss_with_reg(w, EN_train_sentences, trans, emi, EN_all_tags, n = 0.1)
    print('loss: '+ str(loss))
    
    grad_lst = grad_with_reg(ft, w, trans, emi, EN_all_tags, n = 0.1)
    #print('grad_lst: '+ str(grad_lst))
    grads = np.asarray(list(grad_lst)) 
    
    print(' time taken: '+ str(time.time()-start) +' total time: '+ str(time.time()-total_start))
    
    return loss, grads
  
  
init_w = np.asarray([0 for i in range(len(list(index_mapping.keys())))])
result = fmin_l_bfgs_b(get_loss_grad, init_w, pgtol=0.1)

loss: 52775.02915252912
 time taken: 13.905869960784912 total time: 13.935021877288818
loss: 31978.422560595383
 time taken: 13.934737205505371 total time: 27.872556924819946
loss: 19007.65777640145
 time taken: 14.029085159301758 total time: 41.903762102127075
loss: 15909.840926992101
 time taken: 13.852178573608398 total time: 55.75893807411194
loss: 24692.80562540524
 time taken: 13.818942546844482 total time: 69.58005213737488
loss: 12068.697074799862
 time taken: 13.68326449394226 total time: 83.26556944847107
loss: 42879.19468070236
 time taken: 13.560451030731201 total time: 96.82933163642883
loss: 11393.865776589297
 time taken: 13.566591739654541 total time: 110.39802765846252
loss: 10788.155676906375
 time taken: 13.4535653591156 total time: 123.8549256324768
loss: 10197.25111554735
 time taken: 13.52132272720337 total time: 137.38023138046265
loss: 9635.86113042981
 time taken: 13.481806516647339 total time: 150.86600136756897
loss: 8861.897153827815
 time taken: 13.33150076

In [0]:
emi, trans = {}, {}
for i in range(len(list(index_mapping.keys()))): #27899
    if i< len(EN_train_emission): #27818
        emi[index_mapping[i]] = result[0][i]
    else:
        trans[index_mapping[i]] = result[0][i]

write_prediction('data/EN/dev.p4.out', EN_dev_sentences,
                 EN_all_tags, emi, trans)

In [26]:
!python evalResult.py data/EN/dev.out data/EN/dev.p2.out


#Entity in gold data: 210
#Entity in prediction: 154

#Correct Entity : 103
Entity  precision: 0.6688
Entity  recall: 0.4905
Entity  F: 0.5659

#Correct Sentiment : 71
Sentiment  precision: 0.4610
Sentiment  recall: 0.3381
Sentiment  F: 0.3901


In [27]:
!python evalResult.py data/EN/dev.out data/EN/dev.p4.out


#Entity in gold data: 210
#Entity in prediction: 139

#Correct Entity : 104
Entity  precision: 0.7482
Entity  recall: 0.4952
Entity  F: 0.5960

#Correct Sentiment : 73
Sentiment  precision: 0.5252
Sentiment  recall: 0.3476
Sentiment  F: 0.4183


In [0]:
import pickle
f = open("EN_emi.pkl","wb")
pickle.dump(emi,f)
f.close()
f2 = open("EN_trans.pkl","wb")
pickle.dump(emi,f2)

#ff = open('EN_emi.pkl', 'rb')
#ee = pickle.load(ff)
#print(ee)

$\pi(0,y)=a_{START,y}+b_{y}(x_{0})$

$\pi(t,y)=max_{yprev}\left\{\pi(t-1,yprev)+a_{yprev,y}+b_{y}(x_{t}) \right\}$

$max_{y}\left\{\pi(n-1,y)+a_{y,STOP} \right\}$

$\alpha_{y}(0)=a_{START,y}+b_{y}(x_{0})$

$\alpha_{y}(t)=log(\sum_{y\_prev}\left\{exp(\alpha_{y\_prev}(t-1)+a_{y\_prev,y}+b_{y}(x_{t})) \right\})$

$p\_fwd=log(\sum_{y}\left\{exp(\alpha_{y}(n-1)+a_{y,STOP})\right\})$

$\beta_{y}(n-1)=a_{y,STOP}+b_{y}(x_{0})$

$\beta_{y}(t)=log(\sum_{y\_next}\left\{exp(\beta_{y\_next}(t+1)+a_{y,y\_next}+b_{y\_next}(x_{t+1}) )\right\})$

$p\_bkw=log(\sum_{y}\left\{exp(\beta_{y}(0)+a_{START,y} )\right\})$

$E_{(x,y)} = exp(\alpha_{y}(0)+\beta_{y}(0)-p\_fwd)$

$E_{(START,y)} =exp(\beta_{y}(0)+transition(START,y)+emission(x,y)-p\_fwd)$

$E_{(x,y)} = E_{(x,y)}+exp(\alpha_{y}(t)+\beta_{y}(t)-p\_fwd)$

$E_{(y\_prev,y)} = E_{(y\_prev,y)}+exp(\alpha_{y}(t)+\beta_{y}(t)+transition(y\_prev,y)+emission(x,y)-p\_fwd)$

$E_{(y,STOP)} =exp(\alpha_{y}(t)+transition(y\_prev,y)-p\_fwd)$